In [1]:
import os
import pandas as pd
import csv
import time
import matplotlib.pyplot as plt
import numpy as np
import random
import zCurve as z
import hilbert as h
import math
import re

In [2]:
def get_files_in_folder(folder_path):
    try:
        files = [folder_path + "/" + f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
        return files
    except Exception as e:
        print(f"An error occurred while getting files in the folder: {str(e)}")
        return []

def calculate_morton(data_files, scales_dict, offsets_dict):
    data_files.sort()
    data_files.reverse()
    output_dir = "results/real_lanechangecitos_morton"  # Folder to save plots
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # ===============================
    # DATA FILE PROCESSING
    # ===============================
    for file_path in data_files:
        if ".csv" not in file_path:
            continue

        print(file_path)

        # Get the file basename without extension
        file_basename = os.path.basename(file_path)
        results_filepath = os.path.join(output_dir, f"{file_basename}")
        
        data = pd.read_csv(file_path, sep=",")
        data['timestamp'] = data['Time (miliseconds)'] / (10**3)

        # Transformation for speed and steering angle 

        ntimestamps = len(data['timestamp'])
        ndimensions = len(scales_dict)
        tmp_matrix = np.zeros((ntimestamps, ndimensions))

        for j, data_key in enumerate(scales_dict.keys()):
            tmp_matrix[:, j] = [int(i) for i in data[data_key] * scales_dict[data_key] + offsets_dict[data_key]]

        # -------------------------------
        # Build Morton (Z-order) Index (if chosen or if building both)
        # -------------------------------
        data['Morton_Index'] = [z.interlace(int(x), int(y), bits_per_dim=10) for x, y in zip(tmp_matrix[:,0], tmp_matrix[:,1])]
        df = pd.DataFrame(data=data)
        df.to_csv(results_filepath, sep=";", index=False)

    return []

def calculate_hilbert(data_files, scales_dict, offsets_dict):
    data_files.sort()
    data_files.reverse()
    output_dir = "results/real_lanechangecitos_hilbert"  # Folder to save plots
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # ===============================
    # DATA FILE PROCESSING
    # ===============================
    for file_path in data_files:
        if ".csv" not in file_path:
            continue

        print(file_path)

        # Get the file basename without extension
        file_basename = os.path.basename(file_path)
        results_filepath = os.path.join(output_dir, f"{file_basename}")
        
        data = pd.read_csv(file_path, sep=",")
        data['timestamp'] = data['Time (miliseconds)'] / (10**3)

        # Transformation for speed and steering angle 

        ntimestamps = len(data['timestamp'])
        ndimensions = len(scales_dict)
        tmp_matrix = np.zeros((ntimestamps, ndimensions))

        for j, data_key in enumerate(scales_dict.keys()):
            tmp_matrix[:, j] = [int(i) for i in data[data_key] * scales_dict[data_key] + offsets_dict[data_key]]

        # ===============================
        # Build Hilbert Index
        # ===============================
        data["Hilbert_Index"] = h.encode(np.ascontiguousarray(np.array([tmp_matrix[:,0], tmp_matrix[:,1]]).T), 2, 10)
        df = pd.DataFrame(data=data)
        df.to_csv(results_filepath, sep=";", index=False)

    return []

In [3]:
# Set transformation for speed and steering angle
offsets, scales = {"Speed (m/s)": 105}, {"Speed (m/s)": 4}
offsets["Steering angle (deg)"] = 150
scales["Steering angle (deg)"] = 1

# ===============================
# Calculate morton and hilbert
# ===============================

data_files = ['input_data/Lanechangecitos/2018-08-17--14-55-39_7/2018-08-17--14-55-39_7_merged_data_sensor.csv',
                   'input_data/Lanechangecitos/2018-07-27--06-03-57_3/2018-07-27--06-03-57_3_merged_data_sensor.csv']
data_files.sort()
data_files.reverse()
calculate_morton(data_files=data_files,scales_dict=scales, offsets_dict=offsets)
calculate_hilbert(data_files=data_files,scales_dict=scales, offsets_dict=offsets)

input_data/Lanechangecitos/2018-08-17--14-55-39_7/2018-08-17--14-55-39_7_merged_data_sensor.csv
input_data/Lanechangecitos/2018-07-27--06-03-57_3/2018-07-27--06-03-57_3_merged_data_sensor.csv
input_data/Lanechangecitos/2018-08-17--14-55-39_7/2018-08-17--14-55-39_7_merged_data_sensor.csv
input_data/Lanechangecitos/2018-07-27--06-03-57_3/2018-07-27--06-03-57_3_merged_data_sensor.csv


[]